In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras.optimizers import Adam
#from kerastuner import RandomSearch
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import pandas as pd
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import cv2
import numpy as np

train_img = []
train_labels = []

test_img = []
test_labels = []

path_train = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training/'
path_test = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Testing/'
img_size = 300

# Function to process images in a given directory
def process_images(path):
    images = []
    labels = []
    for category in os.listdir(path):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):  # Ensure it's a directory
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    resized_img = cv2.resize(img, (img_size, img_size))
                    images.append(resized_img)
                    labels.append(category)
                else:
                    print(f"Warning: Skipping file {img_path} as it is not a valid image.")
    return images, labels

# Process training and testing data
train_img, train_labels = process_images(path_train)
test_img, test_labels = process_images(path_test)

train_img = np.array(train_img)
test_img = np.array(test_img)

# Encode labels
train_labels_encoded = [0 if category == 'no_tumor' else (1 if category == 'glioma_tumor' else (2 if category == 'meningioma_tumor' else 3)) for category in train_labels]
test_labels_encoded = [0 if category == 'no_tumor' else (1 if category == 'glioma_tumor' else (2 if category == 'meningioma_tumor' else 3)) for category in test_labels]

In [3]:
train_img = tf.cast(train_img, tf.float32)
test_img = tf.cast(test_img, tf.float32)

In [4]:
# One-hot encode labels
train_labels_encoded = np.array(pd.get_dummies(train_labels))
test_labels_encoded = np.array(pd.get_dummies(test_labels))

In [37]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(tf.cast(y_true, tf.float32))
    y_pred_f = tf.keras.backend.flatten(tf.cast(y_pred, tf.float32))
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Define Hausdorff distance
def hausdorff_distance(y_true, y_pred):
    if isinstance(y_true, tf.Tensor):
        y_true = y_true.numpy()
    if isinstance(y_pred, tf.Tensor):
        y_pred = y_pred.numpy()
    
    # Convert to binary masks
    y_true = (y_true > 0.5).astype(int)
    y_pred = (y_pred > 0.5).astype(int)
    
    # Check if masks are 2D
    if y_true.ndim == 1:
        y_true = np.expand_dims(y_true, axis=0)
    if y_pred.ndim == 1:
        y_pred = np.expand_dims(y_pred, axis=0)
    
    # Flatten masks to 1D
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()
    
    # Compute Hausdorff distance
    hd = directed_hausdorff(y_true_flat, y_pred_flat)[0]
    return hd

In [17]:
# Data Augmentation
img_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

img_datagen.fit(train_img)
img_datagen.fit(test_img)

train_generator = img_datagen.flow(train_img, train_labels_encoded, batch_size=32)
validation_generator = img_datagen.flow(test_img, test_labels_encoded, batch_size=32)

In [18]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(kernel_size=(5, 5), filters=32, activation='relu', padding='same', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32, activation='relu', padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(4, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', dice_coefficient, tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanSquaredError()])

In [19]:
# Train the model
history = model.fit(train_generator, 
                    steps_per_epoch=len(train_img) // 32, 
                    epochs=10, 
                    validation_data=validation_generator, 
                    validation_steps=len(test_img) // 32)


Epoch 1/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 81s 874ms/step - accuracy: 0.3419 - dice_coefficient: 0.3592 - loss: 7.0907 - mean_absolute_error: 0.4879 - mean_squared_error: 0.3173 - val_accuracy: 0.2552 - val_dice_coefficient: 0.3347 - val_loss: 1.6595 - val_mean_absolute_error: 0.5131 - val_mean_squared_error: 0.2896
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.4062 - dice_coefficient: 0.2093 - loss: 1.3202 - mean_absolute_error: 0.4740 - mean_squared_error: 0.2800 - val_accuracy: 0.3000 - val_dice_coefficient: 0.1904 - val_loss: 1.4535 - val_mean_absolute_error: 0.4865 - val_mean_squared_error: 0.2696
Epoch 3/10


2024-07-29 01:42:50.274024: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-29 01:42:50.328729: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


89/89 ━━━━━━━━━━━━━━━━━━━━ 79s 859ms/step - accuracy: 0.4066 - dice_coefficient: 0.3925 - loss: 1.2391 - mean_absolute_error: 0.4839 - mean_squared_error: 0.2642 - val_accuracy: 0.3281 - val_dice_coefficient: 0.3431 - val_loss: 1.6343 - val_mean_absolute_error: 0.5310 - val_mean_squared_error: 0.3113
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 665us/step - accuracy: 0.4062 - dice_coefficient: 0.1983 - loss: 1.2679 - mean_absolute_error: 0.4971 - mean_squared_error: 0.2779 - val_accuracy: 0.1000 - val_dice_coefficient: 0.1544 - val_loss: 1.9457 - val_mean_absolute_error: 0.5552 - val_mean_squared_error: 0.3434
Epoch 5/10


2024-07-29 01:44:10.300010: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-29 01:44:10.350880: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


89/89 ━━━━━━━━━━━━━━━━━━━━ 79s 856ms/step - accuracy: 0.4384 - dice_coefficient: 0.4069 - loss: 1.1980 - mean_absolute_error: 0.4843 - mean_squared_error: 0.2763 - val_accuracy: 0.3281 - val_dice_coefficient: 0.3540 - val_loss: 1.7900 - val_mean_absolute_error: 0.5305 - val_mean_squared_error: 0.3298
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 669us/step - accuracy: 0.4375 - dice_coefficient: 0.2042 - loss: 1.2685 - mean_absolute_error: 0.4905 - mean_squared_error: 0.2894 - val_accuracy: 0.4000 - val_dice_coefficient: 0.2069 - val_loss: 1.2941 - val_mean_absolute_error: 0.4943 - val_mean_squared_error: 0.3046
Epoch 7/10


2024-07-29 01:45:30.001525: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-29 01:45:30.052668: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


89/89 ━━━━━━━━━━━━━━━━━━━━ 79s 859ms/step - accuracy: 0.4953 - dice_coefficient: 0.4214 - loss: 1.1317 - mean_absolute_error: 0.4685 - mean_squared_error: 0.2700 - val_accuracy: 0.2500 - val_dice_coefficient: 0.3647 - val_loss: 1.8045 - val_mean_absolute_error: 0.5338 - val_mean_squared_error: 0.3465
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 679us/step - accuracy: 0.4688 - dice_coefficient: 0.2260 - loss: 1.1976 - mean_absolute_error: 0.4575 - mean_squared_error: 0.2947 - val_accuracy: 0.2000 - val_dice_coefficient: 0.2085 - val_loss: 1.4498 - val_mean_absolute_error: 0.5055 - val_mean_squared_error: 0.3415
Epoch 9/10


2024-07-29 01:46:49.908611: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-29 01:46:49.960412: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


89/89 ━━━━━━━━━━━━━━━━━━━━ 79s 861ms/step - accuracy: 0.5123 - dice_coefficient: 0.4440 - loss: 1.0695 - mean_absolute_error: 0.4563 - mean_squared_error: 0.2764 - val_accuracy: 0.3307 - val_dice_coefficient: 0.3719 - val_loss: 1.9957 - val_mean_absolute_error: 0.5168 - val_mean_squared_error: 0.3414
Epoch 10/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 1s 663us/step - accuracy: 0.5625 - dice_coefficient: 0.2538 - loss: 0.7260 - mean_absolute_error: 0.4070 - mean_squared_error: 0.2546 - val_accuracy: 0.2000 - val_dice_coefficient: 0.1307 - val_loss: 3.6418 - val_mean_absolute_error: 0.6172 - val_mean_squared_error: 0.4541


2024-07-29 01:48:10.029493: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-07-29 01:48:10.080303: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [39]:
history = model.fit(tf.cast(train_img,tf.float32),
                    np.array(pd.get_dummies(train_labels)),
                    validation_split=0.1,epochs=20,
                   verbose=1,batch_size = 32)

Epoch 1/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 67s 820ms/step - accuracy: 0.6259 - dice_coefficient: 0.4952 - loss: 0.8584 - mean_absolute_error: 0.4271 - mean_squared_error: 0.2875 - val_accuracy: 0.9477 - val_dice_coefficient: 0.5241 - val_loss: 0.3659 - val_mean_absolute_error: 0.4225 - val_mean_squared_error: 0.2678
Epoch 2/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 68s 834ms/step - accuracy: 0.7453 - dice_coefficient: 0.5344 - loss: 0.5857 - mean_absolute_error: 0.4061 - mean_squared_error: 0.3023 - val_accuracy: 0.9164 - val_dice_coefficient: 0.5811 - val_loss: 0.3038 - val_mean_absolute_error: 0.3414 - val_mean_squared_error: 0.2219
Epoch 3/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 67s 832ms/step - accuracy: 0.8179 - dice_coefficient: 0.5526 - loss: 0.4766 - mean_absolute_error: 0.3851 - mean_squared_error: 0.2987 - val_accuracy: 0.9826 - val_dice_coefficient: 0.5742 - val_loss: 0.1647 - val_mean_absolute_error: 0.3613 - val_mean_squared_error: 0.2352
Epoch 4/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 69s 852ms/step - accura

In [40]:
def evaluate_model(model, test_generator):
    total_hausdorff_distance = 0
    total_samples = 0
    
    for i in range(len(test_generator)):
        x_batch, y_batch = next(test_generator)
        predictions = model.predict(x_batch)
        
        for y_true, y_pred in zip(y_batch, predictions):
            # Convert to binary masks
            y_true = (y_true > 0.5).astype(bool)
            y_pred = (y_pred > 0.5).astype(bool)
            
            # Compute Hausdorff distance
            hd = hausdorff_distance(y_true, y_pred)
            total_hausdorff_distance += hd
            total_samples += 1
    
    return total_hausdorff_distance / total_samples

hausdorff_avg = evaluate_model(model, validation_generator)
print("Average Hausdorff Distance:", hausdorff_avg)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


IndexError: tuple index out of range